In [6]:
from ohlcv_data import OHLCVDataLoader
from momentumStrat import MomentumStratOHLCV
from ohlcv_backtester import OHLCVBacktester
from ohlcv_analysis import OHLCVAnalysis
import os

In [ ]:
# 1. Load and clean data
folder_path = "/Users/elyazidbenkhadra/Downloads/stock_market_data/nyse/csv"
data = OHLCVDataLoader.load_all_data(folder_path, clean=True, years=20)

2025-04-17 21:36:36,908 - INFO - Loaded and combined data from 403 files. Final shape: (4446, 2418)


In [8]:
# 2. Instantiate strategy
strategy = MomentumStratOHLCV(name="Momentum Strategy", short_window =50, long_window=200)

In [9]:
# 3. Run backtest (multi-asset)
backtester = OHLCVBacktester(data, strategy, fee_perc=0.0005)
asset_states, portfolio_equity_curve = backtester.run()

2025-04-17 21:36:46,353 - INFO - Backtest complete. Trades: [28, 16, 24, 16, 16, 18, 26, 26, 26, 22, 23, 27, 26, 30, 28, 27, 21, 30, 27, 26, 24, 12, 8, 24, 18, 20, 37, 20, 22, 14, 28, 20, 25, 26, 24, 26, 22, 20, 27, 18, 26, 23, 26, 24, 32, 30, 28, 22, 27, 3, 26, 24, 26, 26, 24, 32, 26, 32, 30, 26, 30, 20, 27, 29, 26, 20, 30, 24, 26, 23, 18, 24, 31, 14, 24, 28, 28, 28, 17, 21, 32, 22, 26, 20, 25, 28, 18, 27, 25, 12, 26, 26, 24, 23, 7, 24, 20, 31, 28, 26, 20, 15, 28, 30, 26, 18, 20, 22, 29, 26, 19, 24, 20, 24, 26, 28, 20, 32, 22, 24, 22, 26, 27, 22, 20, 28, 26, 18, 19, 26, 15, 26, 14, 30, 18, 22, 26, 20, 18, 24, 26, 16, 16, 18, 19, 6, 16, 10, 24, 27, 19, 29, 34, 24, 29, 22, 23, 28, 14, 16, 31, 25, 28, 29, 18, 16, 18, 25, 15, 8, 32, 28, 14, 16, 35, 22, 26, 24, 25, 29, 25, 22, 28, 27, 28, 24, 5, 20, 22, 28, 8, 36, 32, 23, 22, 22, 26, 20, 19, 28, 22, 22, 33, 3, 20, 7, 8, 14, 20, 30, 24, 22, 30, 32, 18, 28, 32, 24, 25, 28, 18, 24, 28, 24, 26, 24, 18, 18, 20, 18, 25, 26, 25, 28, 28, 24, 27, 2

In [13]:
# 4. Analyze and visualize (multi-asset, portfolio focus)
analysis = OHLCVAnalysis(asset_states, portfolio_equity_curve, data)
analysis.print_portfolio_metrics()

# Plot portfolio equity curve
analysis.plot_portfolio_equity()

# Plot portfolio PnL
analysis.plot_portfolio_pnl()

# Plot portfolio cash and equity allocation
analysis.plot_portfolio_allocations()
analysis.plot_portfolio_cash_cap()
analysis.plot_portfolio_allocations_pct()
analysis.plot_pnl_decomposition()

# (Optional) For a specific asset, you can still use:
loaded_ticker = "AAPL"
analysis.plot_asset_trades(loaded_ticker)
analysis.plot_asset_pnl_with_trades(loaded_ticker)
analysis.plot_asset_equity_allocation_pct(loaded_ticker)
analysis.plot_asset_allocations(loaded_ticker)

Portfolio metrics:
  Sharpe Ratio: 0.38
  Max Drawdown: -48.48%
  Total Return: 327.44%
